In [1]:
#!pip install xformers

In [2]:
from vis_prior_dataset import VisPriorDataset

In [3]:
annotation_path = "/home/ubuntu/dad/roboflow-100-benchmark/rf100/apples-fvpl5/train_aug_bboxhed.json"

In [4]:
# test

dataset = VisPriorDataset(annotation_file=annotation_path)
print(len(dataset))

item = dataset[12]
jpg = item['jpg']
txt = item['txt']
hint = item['hint']
print(txt)
print(jpg.shape)
print(hint.shape)

489
apples
(640, 640, 3)
(640, 640, 3)


In [5]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader
import sys
sys.path.append("/home/ubuntu/dad/ControlNet")
from cldm.logger import ImageLogger
from cldm.model import create_model, load_state_dict


# Configs
model_config = '../../models/cldm_v15.yaml'
resume_path = '../../models/control_sd15_hed.pth'
batch_size = 1
logger_freq = 300
learning_rate = 1e-6
sd_locked = True
only_mid_control = False

#pl


/opt/conda/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [6]:
# First use cpu to load models. Pytorch Lightning will automatically move it to GPUs.
model = create_model(model_config).cpu()
model.load_state_dict(load_state_dict(resume_path, location='cpu'))
model.learning_rate = learning_rate
model.sd_locked = sd_locked
model.only_mid_control = only_mid_control

ControlLDM: Running in eps-prediction mode
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.8.layer_norm1.weight', 'vision_model.encoder.layers.5.layer_norm2.bias', 'vision_model.encoder.layers.0.mlp.fc1.weight', 'vision_model.encoder.layers.6.layer_norm1.weight', 'vision_model.encoder.layers.1.mlp.fc2.bias', 'vision_model.encoder.layers.19.self_attn.out_proj.bias', 'vision_model.encoder.layers.3.mlp.fc2.bias', 'vision_model.encoder.layers.20.layer_norm2.bias', 'vision_model.encoder.layers.7.self_attn.out_proj.weight', 'vision_model.encoder.layers.7.layer_norm2.bias', 'vision_model.encoder.layers.17.self_attn.out_proj.weight', 'visual_projection.weight', 'vision_model.encoder.layers.23.layer_norm2.bias', 'vision_model.encoder.layers.18.layer_norm1.weight', 'vision_model.encoder.layers.11.mlp.fc1.bias', 'vision_model.encoder.layers.14.layer_norm2.weight', 'vision_model.encoder.layers.6.mlp.fc1.weight', 'vision_model.encoder.layers.6

Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is 768 and using 8 heads.
Setting up Me

In [7]:
# Misc
dataloader = DataLoader(dataset, num_workers=0, batch_size=batch_size, shuffle=True)

logger = ImageLogger(batch_frequency=logger_freq)
callbacks_ckpt = pl.callbacks.ModelCheckpoint(every_n_train_steps=1000)

trainer = pl.Trainer(gpus=1, precision=32, callbacks=[logger, callbacks_ckpt], accumulate_grad_batches=4, max_steps=10000)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
# Train!
trainer.fit(model, dataloader)

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:118: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:280: LightningDeprecationWarning: Base `LightningModule.on_train_batch_start` hook signature has changed in v1.5. The `dataloader_idx` argument will be removed in v1.7.
  rank_zero_deprecation(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:287: LightningDeprecationWarning: Base `Callback.on_train_batch_end` hook signature has changed in v1.5. The `dataloader_idx` argument will be removed in v1.7.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name              | Type               | Params
--------------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:56: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Data shape for DDIM sampling is (1, 4, 80, 80), eta 0.0
Running DDIM Sampling with 50 timesteps



DDIM Sampler: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:17<00:00,  2.82it/s]


...


DDIM Sampler: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:17<00:00,  2.82it/s]


Data shape for DDIM sampling is (1, 4, 80, 80), eta 0.0
Running DDIM Sampling with 50 timesteps



DDIM Sampler: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:17<00:00,  2.82it/s]


Data shape for DDIM sampling is (1, 4, 80, 80), eta 0.0
Running DDIM Sampling with 50 timesteps



DDIM Sampler: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:17<00:00,  2.82it/s]
